In [2]:
# automatically load changes from the sandbox_steps module
%load_ext autoreload
%autoreload 2

In [32]:
import sys
import os
from e2b_desktop import Sandbox, CommandExitException
from dotenv import load_dotenv
load_dotenv()

sys.path.append(os.path.abspath('..')) # Add the parent directory (project root) to sys.path

from counter_strike.install_cs import install_cs_1_6
from counter_strike.image_handling import get_screenshot_message, draw_point

from llms.models import AimingModel

E2B_API_KEY = os.environ.get("E2B_API_KEY")
CS_SERVER_IP = os.environ.get("CS_SERVER_IP")


In [307]:
# With custom configuration
desktop = Sandbox(
    display=":0",  # Custom display (defaults to :0)
    resolution=(1920, 1080),  # Custom resolution
    timeout = 3600) 

desktop.stream.start()

# Get stream URL
url = desktop.stream.get_url()
print(url)

# only viewing 
url_view = desktop.stream.get_url(view_only=True)
print(url_view)

https://6080-imuf5225tbcjhut3bj07v-53fc73d7.e2b.app/vnc.html?autoconnect=true&resize=scale
https://6080-imuf5225tbcjhut3bj07v-53fc73d7.e2b.app/vnc.html?autoconnect=true&view_only=true&resize=scale


In [308]:
install_cs_1_6(desktop=desktop)

### Steps to connect to the server

In [309]:
desktop.wait(5000)
desktop.left_click(76, 948) # click on Find Servers
desktop.wait(1000)
desktop.left_click(258, 76) # Click on Favorites Tab
desktop.wait(1000)
desktop.left_click(665, 604) # Add server
desktop.wait(1000)
desktop.write(text=CS_SERVER_IP) # Type server ip Address
desktop.wait(500)
desktop.left_click(1119, 527) # Add server to favourites
desktop.wait(500)
desktop.left_click(273, 126) # Select the added server
desktop.wait(500)
desktop.left_click(849, 606) # Connect
for i in range(5):
    print(i)
    desktop.wait(30_000)


# Now wait at least 2 mins for connection and Download

0
1
2
3
4


In [310]:
# steps to connect to the game
desktop.press("enter")
desktop.wait(300)
desktop.press("1") # 1=T, 2=CT, 6=SPECTATE
desktop.wait(300)
desktop.press("4") # T side: Guerilla warfare skin because they have the red headband

# congrats, you joined the game!!

In [260]:
desktop.move_mouse(150, 10)


In [293]:
screenshot_message = get_screenshot_message(desktop, filename="../images/screenshot.jpg")

In [320]:
desktop.left_click(170, 10)

In [268]:
desktop.press("enter")

In [317]:
from counter_strike.image_handling import get_mouse_movements
aiming_model = AimingModel()

In [ ]:
screenshot_message = get_screenshot_message(desktop, filename="../images/screenshot.jpg")

point_json, _ = aiming_model.complete(messages=screenshot_message)
print(point_json)
coords = aiming_model.parse_point_json(point_json)
print(coords)
print(type(coords))
draw_point(point = coords, image_path="../images/screenshot.jpg", output_path="../images/screenshot_annotated.jpg")

mouse_movements = get_mouse_movements(coords=coords)
def execute_mouse_movements(mouse_movements: List, desktop: "Sandbox"):
    # Execute the movements
    print(f"Aiming based on input coords: {coords}")
    if not mouse_movements:
        print("No movement calculated.")
    else:
        for i, move_coords in enumerate(mouse_movements):
            print(f"Aiming to coords {i+1}: {move_coords}")
            desktop.move_mouse(**move_coords)
    print("-" * 30)

desktop.left_click()
desktop.wait(300)
desktop.left_click()
desktop.wait(300)
desktop.left_click()
desktop.wait(300)

{
  "point": {
    "x": "953",
    "y": "551"
  }
}
{'x': 953, 'y': 551}
<class 'dict'>
Aiming based on input coords: {'x': 953, 'y': 551}
Aiming to coords 1: {'x': 950, 'y': 554}
------------------------------
